In [1]:
from pyspark.sql import SparkSession

In [2]:
SPARK_MASTER="spark://cm1:7077"
APP_NAME="Exemplo Spark, Kafka e ElasticSearch"

KAFKA_HOST="cm3:9092"

KAFKA_TOPIC_IN="student-a190020377-entrada"
KAFKA_TOPIC_OUT="student-a190020377-saida"

INTERVAL = "10 seconds"

### Inicialização da sessão

In [3]:
spark = SparkSession.builder \
        .master(SPARK_MASTER) \
        .appName(APP_NAME) \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/11 00:03:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Inscrição no topico

In [4]:
messages = spark.readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", KAFKA_HOST) \
                .option("subscribe", KAFKA_TOPIC_IN) \
                .option('includeTimestamp', 'true') \
                .load()
messages

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

### Contagem das palavras

In [5]:
from pyspark.sql.functions import explode, split, col, upper, window, to_json, struct, lit

#### Separação das linhas em palavras

In [6]:
words = messages \
            .select(
                explode(split(col("value"), "\s+")).alias("word"),
                messages.timestamp
            ).select(
                upper(col("word")).alias('word'), 
                col("timestamp")
            )
words

DataFrame[word: string, timestamp: timestamp]

#### Agrupamento das palavras

In [7]:
wordCounts = words.withWatermark("timestamp", INTERVAL) \
                    .groupBy(
                        window(words.timestamp, INTERVAL, INTERVAL),
                        "word"
                    ) \
                    .count()
wordCounts

DataFrame[window: struct<start:timestamp,end:timestamp>, word: string, count: bigint]

## Escrita no terminal

In [8]:
wct = wordCounts.writeStream \
                .format('console') \
                .outputMode("update") \
                .start()

23/02/11 00:03:23 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-66b8ef76-e255-4862-bab4-9ada0cc1516a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/02/11 00:03:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+----+-----+
|              window|word|count|
+--------------------+----+-----+
|{2023-02-11 00:03...|  AA|    1|
|{2023-02-11 00:04...|  AA|    1|
+--------------------+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+-----+
|              window| word|count|
+--------------------+-----+-----+
|{2023-02-11 00:04...|SAASD|    1|
|{2023-02-11 00:04...|   AS|    1|
|{2023-02-11 00:04...|    A|    2|
|{2023-02-11 00:04...|ASDAS|    1|
|{2023-02-11 00:04...|    S|    1|
|{2023-02-11 00:04...|     |    1|
|{2023-02-11 00:04...|    D|    1|
|{2023-02-11 00:04...|  BBB|    1|
|{2023-02-11 00:04...|   AA|    2|
+--------------------+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+------+-----+
|              window|  word|count|
+--------------------+------+-----+
|{2023-02-11 00:04...|  ASAC|    1|
|{2023-02-11 00:04...|    AS|    3|
|{2023-02-11 00:04...|ASCASC|    1|
|{2023-02-11 00:04...|   ASC|    2|
|{2023-02-11 00:04...| ASASC|    1|
|{2023-02-11 00:04...|    AC|    1|
|{2023-02-11 00:04...|      |    3|
|{2023-02-11 00:04...|  ASAS|    1|
|{2023-02-11 00:04...|   CAS|    1|
+--------------------+------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+----+-----+
|              window|word|count|
+--------------------+----+-----+
|{2023-02-11 00:04...| ASC|    4|
+--------------------+----+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



## Conversão em JSON

In [9]:
wordCountsJson = wordCounts.select(
                                lit('1').alias('key'),
                                to_json(struct("word", "count")).alias('value')
                            )
wordCountsJson

DataFrame[key: string, value: string]

### Escrita no terminal do JSON

In [10]:
wcj = wordCountsJson.writeStream \
                .format('console') \
                .outputMode("update") \
                .trigger(processingTime=INTERVAL) \
                .start()

23/02/11 00:06:07 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-86d74af9-f8ca-44ce-9e5b-a835cb26fbf2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/02/11 00:06:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+



## Envio para topico do Kafka

In [11]:
wck = wordCountsJson \
        .writeStream \
        .outputMode("update") \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_HOST) \
        .option('topic', KAFKA_TOPIC_OUT) \
        .option('checkpointLocation', '/tmp/spark-a190020377') \
        .trigger(processingTime=INTERVAL) \
        .start()

23/02/11 00:06:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 6
-------------------------------------------
+--------------------+----+-----+
|              window|word|count|
+--------------------+----+-----+
|{2023-02-11 00:06...|   A|    3|
|{2023-02-11 00:06...| AAA|    1|
|{2023-02-11 00:06...|    |    2|
|{2023-02-11 00:06...|  AA|    1|
+--------------------+----+-----+



-------------------------------------------
Batch: 7
-------------------------------------------
+--------------------+----+-----+
|              window|word|count|
+--------------------+----+-----+
|{2023-02-11 00:06...|  AC|    1|
|{2023-02-11 00:06...| ASD|    1|
|{2023-02-11 00:06...|   A|    8|
|{2023-02-11 00:06...|    |    4|
|{2023-02-11 00:06...| SAD|    1|
|{2023-02-11 00:06...|  SD|    1|
|{2023-02-11 00:06...|  AS|    3|
|{2023-02-11 00:06...|   C|    7|
|{2023-02-11 00:06...| DAS|    1|
+--------------------+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+---+--------------------+
|key|               value|
+---+--------------------+
|  1|{"word":"AC","cou...|
|  1|{"word":"QW","cou...|
|  1|{"word":"ASD","co...|
|  1|{"word":"A","coun...|
|  1|{"word":"AAA","co...|
|  1|{"word":"","count...|
|  1|{"word":"SAD","co...|
|  1|{"word":"SD","cou...|
|  1|{"word":"DQW","co...|
|  1|{"word":"AA","cou...|
|  1|{"word":"DADQW","...|
|  1|{"word":"AS","cou...|
|  1|{"word":"C","coun...|
|  1|{"word":"DAS","co...|
+---+--------------------+



-------------------------------------------
Batch: 8
-------------------------------------------
+--------------------+-----+-----+
|              window| word|count|
+--------------------+-----+-----+
|{2023-02-11 00:06...|   QW|    1|
|{2023-02-11 00:06...|  ASD|    2|
|{2023-02-11 00:06...|     |    7|
|{2023-02-11 00:06...|  DQW|    1|
|{2023-02-11 00:06...|DADQW|    1|
|{2023-02-11 00:06...|   AS|    6|
+--------------------+-----+-----+



...

-------------------------------------------
Batch: 32
-------------------------------------------
+--------------------+------+-----+
|              window|  word|count|
+--------------------+------+-----+
|{2023-02-11 00:21...|      |    1|
|{2023-02-11 00:22...|      |    6|
|{2023-02-11 00:22...|   DAS|    1|
|{2023-02-11 00:21...|ASDASD|    1|
|{2023-02-11 00:21...|   SDA|    2|
|{2023-02-11 00:22...|    CC|    3|
|{2023-02-11 00:22...|     C|   21|
|{2023-02-11 00:21...|   ASD|    5|
|{2023-02-11 00:22...|   CCC|    2|
|{2023-02-11 00:22...|   ASD|    1|
+--------------------+------+-----+



-------------------------------------------
Batch: 33
-------------------------------------------
+--------------------+----+-----+
|              window|word|count|
+--------------------+----+-----+
|{2023-02-11 00:22...|   A|   15|
|{2023-02-11 00:22...|    |    3|
+--------------------+----+-----+



-------------------------------------------
Batch: 19
-------------------------------------------
+---+--------------------+
|key|               value|
+---+--------------------+
|  1|{"word":"A","coun...|
|  1|{"word":"","count...|
+---+--------------------+



-------------------------------------------
Batch: 34
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 20
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+



## Finalização

In [10]:
wct.stop()
wcj.stop()
wck.stop

NameError: name 'wct' is not defined

In [11]:
spark.stop()